## Beginning of a Data Story ...

One day, you decided to test your luck by stepping into a venture that was entirely unfamiliar to you. Of course, you didn't venture into it empty-handed, but with a monthly sum of money \$<b>A</b> that you earned from your current business. After allocating a small portion of it to hire a data scientist to optimize your investments ...

## Mathematical Interpretation

This is an extreme case when you don't have any clues about the bound of risks that you may encounter in the venture. What this story brings to you is the reason behind the significant level ($\alpha$) and power ($\beta$) at the mathematical detail. Precisely, $(\alpha, \beta)$ is the solution of this mathematical programing problem: 

$\textrm{minimize}_{(N, \mu_\alpha, \tau)} \quad \tau$

$\textrm{subject to} \quad N C_0 - \kappa C_1 \Phi_{N-1} (\kappa-\frac{\mu_\alpha \sqrt{N}}{\sigma}) \le A$

$\quad N C_0 - \tau' C_1 \Phi_{N-1} (\tau'-\frac{\mu_\alpha \sqrt{N}}{\sigma}) \le -B$

$\quad \kappa \lt 0$

$\quad \gamma \le 0$

$\quad \tau' = \tau - \gamma$


Where $A, B$ is non-negative real numbers. 

In [93]:
import numpy as np
import scipy.stats as sts

def newton_solve_max(f, df, x0, tol, upperbound):
    while True:
        f_crt = f(x0)
        alpha = 1.
        rate = f_crt/df(x0)
        if (x0 - rate)>=upperbound:
            alpha = (x0-upperbound)/rate*.5
        x0 = x0 - rate*alpha
        if abs(f_crt)<tol:
            break
            
    return x0

def solve_max(N, sigma, C0, C1, B): # N*C0-B < 0
    k_max = newton_solve_max(lambda x: (1+1/sts.t.pdf(sts.t.ppf((N*C0-B)/x/C1, N-1),N-1)*((N*C0-B)/x**2/C1)), 
                         lambda x: 1e7*((1+1/sts.t.pdf(sts.t.ppf((N*C0-B)/x/C1, N-1),N-1)*((N*C0-B)/x**2/C1))-(1+1/sts.t.pdf(sts.t.ppf((N*C0-B)/(x-1e-7)/C1, N-1),N-1)*((N*C0-B)/(x-1e-7)**2/C1))),
                         (N*C0-B)/C1-1e-3, 1e-5, (N*C0-B)/C1)
    
    mu_max = sigma/np.sqrt(N)*(k_max-sts.t.ppf((N*C0-B)/k_max/C1, N-1))
    return mu_max
    
N = 3000
sigma = 1e-2
C0 = .5
C1 = 50000.
B0 = 50000.
B1 = 80000.

mu_max = solve_max(N, sigma, C0, C1, B0)

mu_max
# sts.t.sf(mu_max*np.sqrt(N)/sigma,N-1) # mu_max/sigma*np.sqrt(N) # 

-0.00034132391361294093

# Strategies

## Don't Do Meaningless Things

_However, you will see this strategy is not always true. Surprised? :)_

So, what is do meaningless things? You may guess about spending efforts in nothing? That's right! Statistically, it is when your null hypothesis is **H0**: $\mu = 0$ and you want to optimize the probability of Type I error (do something when it actually will return nothing). The solution of the above optimization problem will play the role as a rule of thumb. Thus, the threshold for designing the test will be greater than or equal `mu_max`. In this case, as the threshold larger, the Type I error is smaller. So, if you're a very hard fan of "don't do meaningless things" but you don't have any clues about what is meaningfull things, just "do nothing"! Still want to do something? It leads us to the second strategy ... 

## Be Open for Opportunities. 

This brings you back to the beginning optimization problem. You want to as open as possible for opportunities that you can get back at least a worthy amount \$<b>B</b> from your investment. Thus, you must optimize the half-bounded interval $[\tau,\infty)$ represents for a range of worthy opportunities. 

$\textrm{minimize}_{\mu_\alpha} \quad \mu_\alpha$

$\textrm{subject to} \quad N C_0 - \kappa C_1 \Phi_{N-1} (\kappa-\frac{\mu_\alpha \sqrt{N}}{\sigma}) \le A$

$\quad N C_0 - \tau' C_1 \Phi_{N-1} (\tau'-\frac{\mu_\alpha \sqrt{N}}{\sigma}) \le -B$

$\quad \kappa \lt 0$

$\quad \gamma \le 0$

$\quad \tau' = \tau - \gamma$

In [94]:
def newton_solve_min(f, df, x0, tol, lower_bound):
    while True:
        f_crt = f(x0)
        alpha = 1.
        rate = f_crt/df(x0)
        if (x0 - rate)<=lower_bound:
            alpha = (x0-lower_bound)/rate*.5
        x0 = x0 - rate*alpha
        if abs(f_crt)<tol: # and f_crt>0. and f(x0-tol)<0.:
            break
            
    return x0

def solve_min(N, sigma, C0, C1, B):
    k_min = newton_solve_min(lambda x: sigma/np.sqrt(N)*(x-sts.t.ppf((N*C0-B)/x/C1, N-1))-mu_max, 
                         lambda x: (1.+1./sts.t.pdf(sts.t.ppf((N*C0-B)/x/C1, N-1),N-1)*((N*C0-B)/x**2/C1)),
                         (N*C0-B)/C1+1e-3, 1e-5, (N*C0-B)/C1)
    
    return k_min

k_min = solve_min(N, sigma, C0, C1, -B1)

k_min

1.630471545938515